In [ ]:
from pandas import DataFrame
from DATA225utils import make_connection, dataframe_query

In [ ]:
conn_db = make_connection(config_file = 'bossbunch_db.ini')
cursor_db = conn_db.cursor()

In [ ]:
conn_warehouse = make_connection(config_file = 'bossbunch_wh.ini')
cursor_warehouse = conn_warehouse.cursor()

## Dimesnion Tables

## Orders Table

In [ ]:
sql = ( """
        INSERT INTO bossbunch_wh.Orders(OrderID, 
                                            TotalAmount, Coupon, CustomerID, OrdersKey, Date 
                                          )
            SELECT bossbunch_db.Orders.OrderID, 
                   bossbunch_db.Orders.TotalAmount, 
                   bossbunch_db.Orders.Coupon,
                   bossbunch_db.Orders.CustomerID,
                   bossbunch_db.Orders.Date
            FROM bossbunch_db.Orders
            
        """
      )

cursor_db.execute(sql)
conn_db.commit()

## Customer Table

In [ ]:
sql = ("""
        INSERT INTO bossbunch_wh.Customer(CustomerID, 
                                          CustomerName, 
                                          CustomerAge, 
                                          CustomerEmail, 
                                          CustomerZip, 
                                          CustomerCreditScore, 
                                          RegionID
                                         )
            SELECT bossbunch_db.Customer.CustomerID, 
                   bossbunch_db.Customer.FirstName || ' ' || bossbunch_db.Customer.LastName AS CustomerName,
                   bossbunch_db.Customer.CustomerAge,
                   bossbunch_db.Customer.CustomerEmail,
                   bossbunch_db.Customer.CustomerZip,
                   bossbunch_db.Customer.CustomerCreditScore,
                   bossbunch_db.Customer.RegionID
            FROM bossbunch_db.Customer
       """)

cursor_db.execute(sql)
conn_db.commit()

## Fact Tables

## AnalysisPerCuWyS Fact Table

In [ ]:
sql = ("""
    INSERT IGNORE INTO bossbunch_wh.AnalysisPerCuWyS (SalesAmount, CustomerKey, WineryKey, ShippingKey, OrdersKey)
    SELECT
        bossbunch_wh.SalesPerWOC.RevenueInDollars AS SalesAmount,
        bossbunch_wh.Customer.CustomerKey,
        bossbunch_wh.Winery.WineryKey,
        bossbunch_wh.Shipping.ShippingKey,
        bossbunch_wh.SalesPerWOC.OrdersKey
    FROM
        bossbunch_wh.SalesPerWOC
        INNER JOIN bossbunch_wh.Orders ON bossbunch_wh.Orders.OrdersKey = bossbunch_wh.SalesPerWOC.OrdersKey
        INNER JOIN bossbunch_wh.Customers ON bossbunch_wh.Customers.CustomerID = bossbunch_wh.Orders.CustomerID
        INNER JOIN bossbunch_wh.Winery ON bossbunch_wh.Winery.WineID = bossbunch_wh.SalesPerWOC.WineKey
        INNER JOIN bossbunch_wh.Shipping ON bossbunch_wh.Shipping.ShippingID = bossbunch_wh.Orders.ShippingID
    WHERE NOT EXISTS (
        SELECT 1 FROM bossbunch_wh.AnalysisPerCuWyS
        WHERE bossbunch_wh.AnalysisPerCuWyS.CustomerKey = bossbunch_wh.Customers.CustomerKey
        AND bossbunch_wh.AnalysisPerCuWyS.WineryKey = bossbunch_wh.Winery.WineryKey
        AND bossbunch_wh.AnalysisPerCuWyS.ShippingKey = bossbunch_wh.Shipping.ShippingKey
        AND bossbunch_wh.AnalysisPerCuWyS.OrdersKey = bossbunch_wh.SalesPerWOC.OrdersKey
    )
""")

cursor_warehouse.execute(sql)
conn_warehouse.commit()

## AnalysisPerWF Fact Table

In [ ]:
sql = ("""
    INSERT IGNORE INTO bossbunch_wh.AnalysisPerWF (RatingCount, WineKey, FeedbackKey)
    SELECT
        COUNT(bossbunch_wh.Feedback.Rating) AS RatingCount,
        bossbunch_wh.Wine.WineKey,
        bossbunch_wh.Feedback.FeedbackKey
    FROM
        bossbunch_wh.Feedback
        INNER JOIN bossbunch_wh.Wine ON bossbunch_wh.Wine.WineID = bossbunch_wh.Feedback.WineID
    WHERE NOT EXISTS (
        SELECT 1 FROM bossbunch_wh.AnalysisPerWF
        WHERE bossbunch_wh.AnalysisPerWF.WineKey = bossbunch_wh.Wine.WineKey
        AND bossbunch_wh.AnalysisPerWF.FeedbackKey = bossbunch_wh.Feedback.FeedbackKey
    )
    GROUP BY
        bossbunch_wh.Wine.WineKey, bossbunch_wh.Feedback.FeedbackKey
""")

cursor_warehouse.execute(sql)
conn_warehouse.commit()